In [1]:
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [2]:
import os
print(os.getcwd())  # Shows the current working directory


C:\Users\DELL\NYC Taxi Project\Notebooks


In [3]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [5]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_taxi_data

from_year = 2023
# to_year = datetime.now().year
to_year = 2024
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_taxi_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2023 to 2024
File already exists for 2023-01.
Loading data for 2023-01...
Total records: 3,066,766
Valid records: 2,993,140
Records dropped: 73,626 (2.40%)
Successfully processed data for 2023-01.
File already exists for 2023-02.
Loading data for 2023-02...
Total records: 2,913,955
Valid records: 2,845,058
Records dropped: 68,897 (2.36%)
Successfully processed data for 2023-02.
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 3,403,766
Valid records: 3,331,705
Records dropped: 72,061 (2.12%)
Successfully processed data for 2023-03.
File already exists for 2023-04.
Loading data for 2023-04...
Total records: 3,288,250
Valid records: 3,214,922
Records dropped: 73,328 (2.23%)
Successfully processed data for 2023-04.
File already exists for 2023-05.
Loading data for 2023-05...
Total records: 3,513,649
Valid records: 3,435,875
Records dropped: 77,774 (2.21%)
Successfully processed data for 2023-05.
File already exists for 2023-06.
Loading dat

In [6]:
rides

,pickup_datetime,pickup_location_id
0,2023-01-01 00:32:10,161
1,2023-01-01 00:55:08,43
2,2023-01-01 00:25:04,48
3,2023-01-01 00:03:48,138
4,2023-01-01 00:10:29,107
...,...,...
37463263,2023-12-31 23:04:34,233
37463264,2023-12-31 23:08:15,48
37463265,2023-12-31 23:16:15,196
37463266,2023-12-31 23:21:58,140


In [7]:
rides.shape

(37463268, 2)

In [8]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [9]:
ts_data.shape

(2277600, 3)

In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277600 entries, 0 to 2277599
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 26.1 MB


In [11]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
2277595,2023-12-31 19:00:00,263,188
2277596,2023-12-31 20:00:00,263,198
2277597,2023-12-31 21:00:00,263,232
2277598,2023-12-31 22:00:00,263,160


In [12]:
#!pip install twofish
!pip install hopsworks


In [13]:
import hopsworks
import os

api_key = os.getenv('yaeKKye3HP7lw2kL.5sIZ8NQrVMMY6gvZ3IJQXDeSZnaMe81dRVzGE5SLT04EHxNIlV07pio0o74mv6fA')  
project_name = os.getenv('NYC_TAXI_PROJECT_CDA_P1')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-03-03 11:14:48,256 INFO: Initializing external client
2025-03-03 11:14:48,257 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 11:14:50,486 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215640
Successfully connected to Hopsworks project: None


In [14]:
feature_store = project.get_feature_store()

In [15]:
FEATURE_GROUP_NAME = "nyc_time_hourly"
FEATURE_GROUP_VERSION = 1

In [16]:
import hopsworks 
project = hopsworks.login()
feature_store = project.get_feature_store(name='nyc_taxi_project_cda_p1_featurestore')
feature_group = feature_store.get_feature_group('nyc_time_hourly', version=1)

2025-03-03 11:14:51,091 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 11:14:51,112 INFO: Initializing external client
2025-03-03 11:14:51,114 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 11:14:51,869 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215640


In [17]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=1,
    description="time_series_hourly_feature_group",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

In [18]:
!pip install confluent-kafka

In [19]:
from confluent_kafka import Producer

In [20]:

# Import necessary modules
from confluent_kafka import Producer  # Ensure this import is correct
import hopsworks
import pandas as pd  # Import Pandas for DataFrame creation

# Initialize Hopsworks project and feature store
project = hopsworks.login()
fs = project.get_feature_store()  # Access the feature store from the project

# Define feature group
feature_group = fs.get_or_create_feature_group(
    name="nyc_time_hourly",
    version=1,
    description="Time-series data at hourly frequency",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour"  # Ensure this is defined as TIMESTAMP in the schema
)

# Define your time-series data as a dictionary
ts_data = {
    "pickup_location_id": [1, 2, 3],  # Example data
    "pickup_hour": ["2023-10-01 12:00", "2023-10-01 13:00", "2023-10-01 14:00"],  # Example data as strings
    "value": [100, 150, 200]  # Example data
}

# Convert the dictionary to a Pandas DataFrame
ts_dataframe = pd.DataFrame(ts_data)

# Convert the `pickup_hour` column to a datetime object
ts_dataframe["pickup_hour"] = pd.to_datetime(ts_dataframe["pickup_hour"])

# Insert data into the feature group
feature_group.insert(ts_dataframe, write_options={"wait_for_job": False})

2025-03-03 11:15:03,618 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 11:15:03,632 INFO: Initializing external client
2025-03-03 11:15:03,633 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 11:15:04,489 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215640


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 3/3 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: nyc_time_hourly_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215640/jobs/named/nyc_time_hourly_1_offline_fg_materialization/executions


(Job('nyc_time_hourly_1_offline_fg_materialization', 'SPARK'), None)

In [21]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

DataFrame size: 857.47 MB
